In [1]:
import pandas as pd
import io
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib
import matplotlib.pyplot as plt 


In [2]:
#!pip install numpy

In [3]:
#url = 'https://raw.githubusercontent.com/mcelikkaya/medium_articles2/main/user_movie_rating.csv'
#df_user_movie_rating = pd.read_csv(url)

#url = 'https://raw.githubusercontent.com/mcelikkaya/medium_articles2/main/user_movie_rating.csv'
df_user_movie_rating = pd.read_csv("D:/user_movie_rating.csv")

In [4]:
len(df_user_movie_rating)

105339

In [5]:
#df_user_movie_rating.drop(df_user_movie_rating.columns[0])
#df_user_movie_rating = df_user_movie_rating[["userId","movieId","rating","title","genres"]]

In [6]:
#df_user_movie_rating.to_csv("D:/user_movie_rating.csv")

In [7]:
df_user_movie_rating.head(5)

,userId,movieId,rating,title,genres
0,1,16,4.0,Casino (1995),Crime|Drama
1,9,16,4.0,Casino (1995),Crime|Drama
2,12,16,1.5,Casino (1995),Crime|Drama
3,24,16,4.0,Casino (1995),Crime|Drama
4,29,16,3.0,Casino (1995),Crime|Drama


In [8]:
#checks unique values
df_user_movie_rating.nunique()

userId       668
movieId    10325
rating        10
title      10323
genres       938
dtype: int64

In [9]:
#get the max id in set, and add yourself as next user
max_id = max(df_user_movie_rating["userId"])
print("max_id :",max_id)
my_id = max_id + 1
print("my_id :",my_id)

max_id : 668
my_id : 669


In [10]:
#create a map of movieId -> title
title_movieId_map = { y:x for x,y in df_user_movie_rating[ ["movieId","title"] ].to_numpy() }
#'Casino (1995)': 16,
# 'Powder (1995)': 24,

In [11]:
#defines movies that you want to rate 5 or whatever u want (change the 5 at 1st loop )
my_5_list = ['Braveheart (1995)','Seven (a.k.a. Se7en) (1995)','Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
             'Star Wars: Episode VI - Return of the Jedi (1983)','Seven Samurai (Shichinin no samurai) (1954)',
             'Shrek 2 (2004)','Truman Show, The (1998)','Terminator, The (1984)','Rocky (1976)','Mummy, The (1999)'
             ]

#defines movies that you want to rate 2 or whatever u want (change the 2 at 2nd loop )
my_2_list = [ 'Dumb & Dumber (Dumb and Dumber) (1994)','Kill Bill: Vol. 2 (2004)','Space Jam (1996)' ]

for my_5 in my_5_list:
    movide_id = title_movieId_map[my_5] 
    df_user_movie_rating.loc[len(df_user_movie_rating)] = [my_id, movide_id, 5,my_5,"nogenre"] 

for my_2 in my_2_list:
    movide_id = title_movieId_map[my_2] 
    df_user_movie_rating.loc[len(df_user_movie_rating)] = [my_id, movide_id, 2,my_2,"nogenre"] 

In [12]:
#check your ratings
df_user_movie_rating[df_user_movie_rating["userId"]==my_id ]

,userId,movieId,rating,title,genres
105339,669,110,5.0,Braveheart (1995),nogenre
105340,669,47,5.0,Seven (a.k.a. Se7en) (1995),nogenre
105341,669,32,5.0,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),nogenre
105342,669,1210,5.0,Star Wars: Episode VI - Return of the Jedi (1983),nogenre
105343,669,2019,5.0,Seven Samurai (Shichinin no samurai) (1954),nogenre
105344,669,8360,5.0,Shrek 2 (2004),nogenre
105345,669,1682,5.0,"Truman Show, The (1998)",nogenre
105346,669,1240,5.0,"Terminator, The (1984)",nogenre
105347,669,1954,5.0,Rocky (1976),nogenre
105348,669,2617,5.0,"Mummy, The (1999)",nogenre


In [35]:
#create utility maps
movieId_genres_map = { x:y for x,y in df_user_movie_rating[ ["movieId","genres"] ].to_numpy() }
movieId_title_map = { x:y for x,y in df_user_movie_rating[ ["movieId","title"] ].to_numpy() }

genres_sorted = [movieId_genres_map[k] for k in sorted(movieId_genres_map.keys())]
title_sorted = [movieId_title_map[k] for k in sorted(movieId_title_map.keys())]

index_movieId_map = { index:k for index,k in enumerate(sorted(movieId_genres_map.keys())) }
movieId_index_map = { index_movieId_map[k]:k for k in index_movieId_map.keys() }

print( len(movieId_title_map) )
print(title_sorted[0:4])
print(movieId_title_map[1] )
print(movieId_title_map[2] )

10325
['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)', 'Waiting to Exhale (1995)']
Toy Story (1995)
Jumanji (1995)


In [13]:
from scipy.sparse.linalg import svds
import numpy as np

In [14]:
from collections import defaultdict
from surprise import SVD
from surprise import Reader, Dataset, SVD, SVDpp
from surprise import accuracy
from surprise import  KNNWithMeans, KNNWithZScore, CoClustering


In [15]:
len(df_user_movie_rating)

105352

In [16]:
df_user_movie_rating[ df_user_movie_rating["userId"] == my_id ]

,userId,movieId,rating,title,genres
105339,669,110,5.0,Braveheart (1995),nogenre
105340,669,47,5.0,Seven (a.k.a. Se7en) (1995),nogenre
105341,669,32,5.0,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),nogenre
105342,669,1210,5.0,Star Wars: Episode VI - Return of the Jedi (1983),nogenre
105343,669,2019,5.0,Seven Samurai (Shichinin no samurai) (1954),nogenre
105344,669,8360,5.0,Shrek 2 (2004),nogenre
105345,669,1682,5.0,"Truman Show, The (1998)",nogenre
105346,669,1240,5.0,"Terminator, The (1984)",nogenre
105347,669,1954,5.0,Rocky (1976),nogenre
105348,669,2617,5.0,"Mummy, The (1999)",nogenre


In [17]:
#Just give your scale and columns
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(df_user_movie_rating[['userId', 'movieId', 'rating']], reader=reader)



In [18]:
trainset = dataset.build_full_trainset()


<class 'surprise.trainset.Trainset'>


In [73]:
# list of ratings that can be used as a testset in the
#The ratings are all the ratings that are **not** in the trainset
#so this set = user_count * item_count - already_rated
anti_testset = trainset.build_anti_testset()
len(anti_testset )

6802073

In [20]:
#The ratings are all the ratings that are in the trainset,
#already_rated , length of original df_user_movie_rating dataframe
testset = trainset.build_testset()
len(testset )

105352

In [21]:
#check my items in set
trainset.ur[ trainset.to_inner_uid(my_id) ]

[(5, 5.0),
 (3, 5.0),
 (2, 5.0),
 (43, 5.0),
 (254, 5.0),
 (328, 5.0),
 (352, 5.0),
 (417, 5.0),
 (431, 5.0),
 (438, 5.0),
 (298, 2.0),
 (377, 2.0),
 (408, 2.0)]

In [22]:
#Library has a concept of inner_id and your original id
#check methods for which to use
trainset.to_inner_uid(my_id)

668

In [23]:
#get the items rated and NOT rated by user and return them
def get_user_sets(user_id):
    inner_user_id = user_id 
    blank_user_set = []
    for uid, iid, fill in anti_testset:
        if uid == inner_user_id:
            blank_user_set.append( ( uid, iid, fill ) )
    rated_user_set = []        
    for uid, iid, fill in testset:
        if uid == inner_user_id:
            rated_user_set.append( ( uid, iid, fill ) )        
            
    return  blank_user_set  ,rated_user_set     
    

In [24]:
user_blank_set,rated_user_set =  get_user_sets( my_id )

In [25]:
#check if numbers are correct
print(len(user_blank_set))
print(len(rated_user_set))

10312
13


In [26]:
#see a sample
rated_user_set[0]

(669, 110, 5.0)

In [27]:
#try different algorithms with 1 line  :(
#they will not need us anymore
svd_100 = SVD(n_factors=100)
svd_100.fit(trainset)

svd_20 = SVD(n_factors=20)
svd_20.fit(trainset)

svd_50 = SVD(n_factors=50)
svd_50.fit(trainset)

knn_WithMeans = KNNWithMeans()
knn_WithMeans.fit(trainset)

knn_WithZScore = KNNWithZScore()
knn_WithZScore.fit(trainset)

co_Clustering = CoClustering()
co_Clustering.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


C:\Users\mcelikkaya\Anaconda3\envs\neuref5\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [28]:
#Time consuming do not run if you cannot wait
svd_plusplus = SVDpp(n_factors=50)
svd_plusplus.fit(trainset)

In [30]:
#find predictions and calculate rmse,mae
def get_predictions(model,dataset):
    preds = model.test(dataset)    
    rmse = accuracy.rmse(preds, verbose=False)
    mae = accuracy.mae(preds, verbose=False)
        
    preds.sort(key=lambda x: x[3], reverse=True)    
    preds = preds[0:20]
    df = pd.DataFrame(preds,columns=["userId","movieId","actual","predicted","details"])    
    df["movieId"] = df["movieId"].apply(lambda x : movieId_title_map[int(x)] )
    
    return df[ ["movieId","actual","predicted"] ],rmse,mae

In [36]:
#check one sample , we can call like this to get rmse and mae
_,rmse,mae = get_predictions(svd_100,rated_user_set)
print(f"rmse :{round(rmse,3)} \nmae  :{round(mae,3)}")

rmse :0.851 
mae  :0.758


In [37]:
#ugly method instead of toString
def get_obj_name(obj_name):
    ret_name = ""
    for objname, oid in globals().items():
        if oid is obj_name:
            #print("objname, oid",objname, oid)
            ret_name = objname
    return ret_name        

In [60]:
len(rated_user_set)

13

In [59]:
#Evaluate all models and dump their Rmse and Mae
#This is for me, I am checking best results for me, 
#but this is not reliable, dataset is so small

user_blank_set,rated_user_set =  get_user_sets( my_id )

models_to_eval = [svd_20,svd_50,svd_100 ,svd_plusplus, knn_WithMeans, knn_WithZScore, co_Clustering]
models_names = ["svd_20","svd_50","svd_100","svd_plusplus","knn_WithMeans","knn_WithZScore","co_Clustering"]


results = []
for model_,name_ in zip( models_to_eval,models_names ) :
    _,rmse,mae = get_predictions(model_,rated_user_set)
    results.append( [name_,round(rmse,3),round(mae,3) ]) 
    
pd.DataFrame(results,columns = ["Model Name","Rmse","Mae"]).sort_values(by="Mae")    

,Model Name,Rmse,Mae
5,knn_WithZScore,0.732,0.380
4,knn_WithMeans,0.818,0.476
3,svd_plusplus,0.844,0.700
2,svd_100,0.851,0.758
6,co_Clustering,1.060,0.766
1,svd_50,0.956,0.825
0,svd_20,1.030,0.907


In [61]:
#Evaluate all models and dump their Rmse and Mae
#This is for all set so more reliable
models_to_eval = [svd_20,svd_50,svd_100 ,svd_plusplus, knn_WithMeans, knn_WithZScore, co_Clustering]
models_names = ["svd_20","svd_50","svd_100","svd_plusplus","knn_WithMeans","knn_WithZScore","co_Clustering"]


results = []
for model_,name_ in zip( models_to_eval,models_names ) :
    _,rmse,mae = get_predictions(model_,testset)
    results.append( [name_,round(rmse,3),round(mae,3) ]) 
    
pd.DataFrame(results,columns = ["Model Name","Rmse","Mae"]).sort_values(by="Mae") 

,Model Name,Rmse,Mae
3,svd_plusplus,0.581,0.450
2,svd_100,0.635,0.496
5,knn_WithZScore,0.686,0.511
4,knn_WithMeans,0.690,0.517
1,svd_50,0.707,0.550
0,svd_20,0.766,0.594
6,co_Clustering,0.820,0.640


In [62]:
df,_,_ = get_predictions(svd_100,user_blank_set)
df.head(10)

,movieId,actual,predicted
0,Schindler's List (1993),3.516948,5.000000
1,"Godfather, The (1972)",3.516948,4.946849
2,"Godfather: Part II, The (1974)",3.516948,4.912625
3,Touch of Evil (1958),3.516948,4.901958
4,"Treasure of the Sierra Madre, The (1948)",3.516948,4.892755
5,One Flew Over the Cuckoo's Nest (1975),3.516948,4.890692
6,Forrest Gump (1994),3.516948,4.878272
7,North by Northwest (1959),3.516948,4.847157
8,Cinema Paradiso (Nuovo cinema Paradiso) (1989),3.516948,4.840283
9,M (1931),3.516948,4.834478


In [41]:
df,_,_ = get_predictions(knn_WithZScore,user_blank_set)
df.head(10)

,movieId,actual,predicted
0,"Usual Suspects, The (1995)",3.516948,5
1,"Shawshank Redemption, The (1994)",3.516948,5
2,"Fugitive, The (1993)",3.516948,5
3,Schindler's List (1993),3.516948,5
4,Terminator 2: Judgment Day (1991),3.516948,5
5,Fargo (1996),3.516948,5
6,"Godfather, The (1972)",3.516948,5
7,Casablanca (1942),3.516948,5
8,Night of the Living Dead (1968),3.516948,5
9,Monty Python and the Holy Grail (1975),3.516948,5


In [63]:
df,_,_ = get_predictions(svd_plusplus,user_blank_set)
df.head(10)

,movieId,actual,predicted
0,Schindler's List (1993),3.516948,5.000000
1,American Beauty (1999),3.516948,5.000000
2,"Usual Suspects, The (1995)",3.516948,4.993633
3,"Shawshank Redemption, The (1994)",3.516948,4.946595
4,Shakespeare in Love (1998),3.516948,4.870840
5,Fargo (1996),3.516948,4.860878
6,Raiders of the Lost Ark (Indiana Jones and the...,3.516948,4.860418
7,Cinema Paradiso (Nuovo cinema Paradiso) (1989),3.516948,4.851839
8,"Lord of the Rings: The Return of the King, The...",3.516948,4.847290
9,Nausica? of the Valley of the Wind (Kaze no ta...,3.516948,4.826429


In [51]:
df,_,_ = get_predictions(svd_20,user_blank_set)
df.head(10)

,movieId,actual,predicted
0,Citizen Kane (1941),3.516948,4.895400
1,Ran (1985),3.516948,4.886316
2,Nausica? of the Valley of the Wind (Kaze no ta...,3.516948,4.878094
3,"Shawshank Redemption, The (1994)",3.516948,4.856277
4,"Usual Suspects, The (1995)",3.516948,4.855706
5,"Third Man, The (1949)",3.516948,4.844790
6,Touch of Evil (1958),3.516948,4.843602
7,Princess Mononoke (Mononoke-hime) (1997),3.516948,4.836273
8,"Lives of Others, The (Das leben der Anderen) (...",3.516948,4.836166
9,"Godfather, The (1972)",3.516948,4.834204


In [52]:
df,_,_ = get_predictions(svd_100,user_blank_set)
df.head(10)

,movieId,actual,predicted
0,Schindler's List (1993),3.516948,5.000000
1,"Godfather, The (1972)",3.516948,4.946849
2,"Godfather: Part II, The (1974)",3.516948,4.912625
3,Touch of Evil (1958),3.516948,4.901958
4,"Treasure of the Sierra Madre, The (1948)",3.516948,4.892755
5,One Flew Over the Cuckoo's Nest (1975),3.516948,4.890692
6,Forrest Gump (1994),3.516948,4.878272
7,North by Northwest (1959),3.516948,4.847157
8,Cinema Paradiso (Nuovo cinema Paradiso) (1989),3.516948,4.840283
9,M (1931),3.516948,4.834478


In [53]:
df,_,_ = get_predictions(svd_plusplus,rated_user_set)
df.head(10)

,movieId,actual,predicted
0,Braveheart (1995),5.0,4.832746
1,Seven Samurai (Shichinin no samurai) (1954),5.0,4.791703
2,"Truman Show, The (1998)",5.0,4.734876
3,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),5.0,4.562267
4,"Terminator, The (1984)",5.0,4.561600
5,Seven (a.k.a. Se7en) (1995),5.0,4.496032
6,Star Wars: Episode VI - Return of the Jedi (1983),5.0,4.476152
7,Shrek 2 (2004),5.0,4.294429
8,Rocky (1976),5.0,4.191113
9,Kill Bill: Vol. 2 (2004),2.0,3.990636


In [54]:
df,_,_ = get_predictions(svd_plusplus,user_blank_set)
df.head(10)

,movieId,actual,predicted
0,Schindler's List (1993),3.516948,5.000000
1,American Beauty (1999),3.516948,5.000000
2,"Usual Suspects, The (1995)",3.516948,4.993633
3,"Shawshank Redemption, The (1994)",3.516948,4.946595
4,Shakespeare in Love (1998),3.516948,4.870840
5,Fargo (1996),3.516948,4.860878
6,Raiders of the Lost Ark (Indiana Jones and the...,3.516948,4.860418
7,Cinema Paradiso (Nuovo cinema Paradiso) (1989),3.516948,4.851839
8,"Lord of the Rings: The Return of the King, The...",3.516948,4.847290
9,Nausica? of the Valley of the Wind (Kaze no ta...,3.516948,4.826429


In [55]:
df,_,_ = get_predictions(svd_50,user_blank_set)
df.head(10)

,movieId,actual,predicted
0,Touch of Evil (1958),3.516948,4.969714
1,Princess Mononoke (Mononoke-hime) (1997),3.516948,4.937062
2,Citizen Kane (1941),3.516948,4.922594
3,Spirited Away (Sen to Chihiro no kamikakushi) ...,3.516948,4.873472
4,Monty Python and the Holy Grail (1975),3.516948,4.854978
5,Star Wars: Episode V - The Empire Strikes Back...,3.516948,4.843778
6,"Shawshank Redemption, The (1994)",3.516948,4.820622
7,Harold and Maude (1971),3.516948,4.815648
8,"Godfather, The (1972)",3.516948,4.798313
9,Intouchables (2011),3.516948,4.786998


In [56]:
df,_,_ = get_predictions(svd_50,rated_user_set)
df.head(10)

,movieId,actual,predicted
0,Seven (a.k.a. Se7en) (1995),5.0,4.705617
1,Seven Samurai (Shichinin no samurai) (1954),5.0,4.678926
2,Braveheart (1995),5.0,4.578874
3,Star Wars: Episode VI - Return of the Jedi (1983),5.0,4.527020
4,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),5.0,4.424498
5,"Terminator, The (1984)",5.0,4.415782
6,"Truman Show, The (1998)",5.0,4.361911
7,Shrek 2 (2004),5.0,4.167126
8,Kill Bill: Vol. 2 (2004),2.0,4.101751
9,Rocky (1976),5.0,4.033861


In [57]:
df,_,_ = get_predictions(svd_plusplus,user_blank_set)
df.head(10)

,movieId,actual,predicted
0,Schindler's List (1993),3.516948,5.000000
1,American Beauty (1999),3.516948,5.000000
2,"Usual Suspects, The (1995)",3.516948,4.993633
3,"Shawshank Redemption, The (1994)",3.516948,4.946595
4,Shakespeare in Love (1998),3.516948,4.870840
5,Fargo (1996),3.516948,4.860878
6,Raiders of the Lost Ark (Indiana Jones and the...,3.516948,4.860418
7,Cinema Paradiso (Nuovo cinema Paradiso) (1989),3.516948,4.851839
8,"Lord of the Rings: The Return of the King, The...",3.516948,4.847290
9,Nausica? of the Valley of the Wind (Kaze no ta...,3.516948,4.826429


In [58]:
df,_,_ = get_predictions(svd_plusplus,rated_user_set)
df.head(10)

,movieId,actual,predicted
0,Braveheart (1995),5.0,4.832746
1,Seven Samurai (Shichinin no samurai) (1954),5.0,4.791703
2,"Truman Show, The (1998)",5.0,4.734876
3,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),5.0,4.562267
4,"Terminator, The (1984)",5.0,4.561600
5,Seven (a.k.a. Se7en) (1995),5.0,4.496032
6,Star Wars: Episode VI - Return of the Jedi (1983),5.0,4.476152
7,Shrek 2 (2004),5.0,4.294429
8,Rocky (1976),5.0,4.191113
9,Kill Bill: Vol. 2 (2004),2.0,3.990636
